In [1]:
import pandas as pd
import numpy as np
import re
import tqdm

from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import spacy



from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split

import benepar

/home/yashaswi.pathak/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
stop_words= set(stopwords.words('english'))
spacy.prefer_gpu()
nlp=spacy.load('en_core_web_sm')


In [10]:
doc = nlp(u'A woman is walking through the door.')
for token in doc:
    print("{2}({3}-{6}, {0}-{5})".format(token.text, token.tag_, token.dep_, token.head.text, token.head.tag_, token.i+1, token.head.i+1))

det(woman-2, A-1)
nsubj(walking-4, woman-2)
aux(walking-4, is-3)
ROOT(walking-4, walking-4)
prep(walking-4, through-5)
det(door-7, the-6)
pobj(through-5, door-7)
punct(walking-4, .-8)


In [89]:
dataset=pd.read_csv('data/dataset.csv')

In [90]:
dataset.head()

,id,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [93]:

def clean_tweet(tweet):
    tweet = re.sub("#", "",tweet) # Removing '#' from hashtags
    tweet = re.sub("RT","",tweet)
    tweet = re.sub("[@][a-zA-Z0-9]*"," ",tweet)
    
    tweet = re.sub("[^a-zA-Z#]", " ",tweet) # Removing punctuation and special characters
    tweet = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',"<URL>", tweet)
    tweet = re.sub('http','',tweet)
    tweet = re.sub(" +", " ", tweet)
    tweet = tweet.lower()
    return tweet

In [94]:
dataset["cleaned_tweet"]=dataset["tweet"].apply(clean_tweet)

In [65]:
parser = benepar.Parser("benepar_en2")

In [97]:
tree = parser.parse(dataset['cleaned_tweet'].values[0])

In [98]:
tree.pretty_print()

                                                                         S                                                               
      ___________________________________________________________________|________                                                        
     |             |                                                              VP                                                     
     |             |      ________________________________________________________|___                                                    
     |             |     |     |                                                      VP                                                 
     |             |     |     |              ________________________________________|_____________________________                      
     |             |     |     |             VP                                                   |                 |                    
     |             |     |     

In [95]:
dataset["tweet"].values[0]

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [96]:
dataset["cleaned_tweet"].values[0]

' as a woman you shouldn t complain about cleaning up your house amp as a man you should always take the trash out '